In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Képek mérete és batch méret
IMAGE_SHAPE = (200, 200)
BATCH_SIZE = 5

# Adatok elérési útvonalai
train_dir = "drive/MyDrive/dataset/train/"
test_dir = "drive/MyDrive/dataset/test/"

# Kép adatgenerátorok létrehozása
train_datagen = ImageDataGenerator(rescale=1 / 255.)
test_datagen = ImageDataGenerator(rescale=1 / 255.)

# Tanító adathalmaz betöltése
print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

# Teszt adathalmaz betöltése
print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

# Konvolúciós neurális háló létrehozása
model = tf.keras.models.Sequential([
    layers.Conv2D(256, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(momentum=0.9),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(2048, activation='relu'),
    layers.Dropout(0.9),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

# Modell összefoglalása
model.summary()

# Optimalizáló inicializálása és modell összeállítása
initial_learning_rate = 0.00001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=50, decay_rate=0.9, staircase=True)
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.AdamW(learning_rate=lr_schedule),
              metrics=['accuracy'])

# Bővített adathalmaz generátor létrehozása
train_datagen_augmented = ImageDataGenerator(
    rescale=1 / 255.,
    rotation_range=40,
    shear_range=0.3,
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Bővített tanító adathalmaz generálása
augmented_train_data = train_datagen_augmented.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

# Tanító adathalmazok összekapcsolása
combined_train_data = tf.data.Dataset.from_generator(
    lambda: train_data,
    output_signature=(
        tf.TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.float32)
    )
).concatenate(
    tf.data.Dataset.from_generator(
        lambda: augmented_train_data,
        output_signature=(
            tf.TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 2), dtype=tf.float32)
        )
    )
)

# Fake tanítás
model.fit(combined_train_data,
          epochs=1,
          steps_per_epoch=len(train_data) + len(augmented_train_data),
          validation_data=test_data,
          validation_steps=len(test_data))

model.save("PheoModel.h5")

# Korai leállítás és modell mentés callback-ek inicializálása
early_stopping = EarlyStopping(monitor='val_accuracy',
                               patience=120,
                               restore_best_weights=True,
                               verbose=1)
model_checkpoint = ModelCheckpoint("PheoModel.h5",
                                   save_best_only=True,
                                   monitor='val_accuracy',
                                   mode='max',
                                   verbose=1)
# Modell tanítása
loaded_model = load_model("PheoModel.h5")
history = loaded_model.fit(combined_train_data,
                           epochs=560,
                           steps_per_epoch=10,
                           validation_data=test_data,
                           validation_steps=len(test_data),
                           callbacks=[early_stopping, model_checkpoint])


# Loss görbék ábrázolása
def plot_loss_curves(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    epochs = range(len(history.history['loss']))

    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()


plot_loss_curves(history)

# Legjobb modell betöltése és pontosság kiírása
model2 = tf.keras.models.load_model("PheoModel.h5")
print("Legjobb modell pontossága:", max(history.history['val_accuracy']))


Training images:
Found 1000 images belonging to 2 classes.
Testing images:
Found 624 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 256)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 128)       295040    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 128)       0         
 g2D)                                                            
                                                                 
 batch_normalization (Batch  (None, 48, 48, 128)       512       
 Norma

KeyboardInterrupt: ignored